In [1]:
import pandas as pd
import numpy as np

In [2]:
events = pd.read_csv('events.csv')
gold= pd.read_csv('gold.csv')
heroes = pd.read_csv('heroes.csv')
items= pd.read_csv('items.csv')
lh = pd.read_csv('lh.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
xp = pd.read_csv('xp.csv')

In [3]:
xp['xp_radiant'] = xp.loc[:, 'player_0':'player_4'].sum(axis=1)
xp['xp_dire'] = xp.loc[:, 'player_5':'player_9'].sum(axis=1)
xp_team = pd.DataFrame(xp['mid'].unique(), columns=['mid'])
xp_team[['xp_radiant','xp_dire']] = xp.groupby('mid')[['xp_radiant','xp_dire']].sum()


In [4]:
gold['gold_radiant'] = gold.loc[:, 'player_0':'player_4'].sum(axis=1)
gold['gold_dire'] = gold.loc[:, 'player_5':'player_9'].sum(axis=1)
gold_team = pd.DataFrame(gold['mid'].unique(), columns=['mid'])
gold_team[['gold_radiant','gold_dire']] = gold.groupby('mid')[['gold_radiant','gold_dire']].sum()
# gold_team.to_csv('gold_team.csv')

In [5]:
items['items'] = items.loc[:, 'item_0':'item_120'].sum(axis=1)
items_team = pd.DataFrame(items['mid'].unique(), columns=['mid'])
items_team['items_all'] = items.groupby('mid')[['items']].sum()
items_team['items_radiant'] = items.loc[items['player'].astype(int) < 5].groupby('mid')['items'].sum()
items_team['items_dire'] = items.loc[items['player'].astype(int) >= 5].groupby('mid')['items'].sum()
items_team.drop('items_all', axis=1, inplace=True)
# items_team.to_csv('items_team.csv')

In [6]:
lh['lh_radiant'] = lh.loc[:, 'player_0':'player_4'].sum(axis=1)
lh['lh_dire'] = lh.loc[:, 'player_5':'player_9'].sum(axis=1)
lh_team = pd.DataFrame(lh['mid'].unique(), columns=['mid'])
lh_team[['lh_radiant','lh_dire']] = lh.groupby('mid')[['lh_radiant','lh_dire']].sum()
# lh_team.to_csv('lh_team.csv')

In [92]:
# xp_team.to_csv('xp_team.csv')
# gold_team.to_csv('gold_team.csv')
# items_team.to_csv('items_team.csv')
# lh_team.to_csv('lh_team.csv')

In [94]:
# xp_team = pd.read_csv('xp_team.csv')

# gold_team = pd.read_csv('gold_team.csv')
# items_team = pd.read_csv('items_team.csv')
# lh_team = pd.read_csv('lh_team.csv')


In [150]:
events = pd.get_dummies(columns=['event_type'], data=events,dtype=int)

In [151]:
# Группировка по 'mid' и суммирование значений для каждого event_type столбца
event_columns = [col for col in events.columns if 'event_type' in col]
events_radiant = events[events['from_team']=='radiant'].groupby('mid')[event_columns].sum().reset_index()

# Переименуем столбцы, чтобы получить 'count' вместо 'event_type'
events_radiant.columns = ['mid'] + [f'count_radiant_{col}' for col in event_columns]


In [152]:
event_columns = [col for col in events.columns if 'event_type' in col]
events_dire = events[events['from_team']=='dire'].groupby('mid')[event_columns].sum().reset_index()
events_dire.columns = ['mid'] + [f'count_dire_{col}' for col in event_columns]

In [153]:
events_teams = pd.concat([events_radiant, events_dire])
events_teams = events_teams.sort_values(by='mid').reset_index(drop=True)
events_teams.fillna(0, inplace=True)
events_teams=events_teams.groupby('mid',as_index=False).max()

In [154]:
data_train = pd.merge(train, xp_team, on='mid', how='left')
data_train = pd.merge(data_train, gold_team, on='mid', how='left')
data_train = pd.merge(data_train, items_team, on='mid', how='left')
data_train = pd.merge(data_train, lh_team, on='mid', how='left')
data_train = pd.merge(data_train, events_teams, on='mid', how='left')


In [155]:
data_test = pd.merge(test, xp_team, on='mid', how='left')
data_test = pd.merge(data_test, gold_team, on='mid', how='left')
data_test = pd.merge(data_test, items_team, on='mid', how='left')
data_test = pd.merge(data_test, lh_team, on='mid', how='left')

data_test = pd.merge(data_test, events_teams, on='mid',how='left')

In [156]:
data_train['dif_xp'] = data_train['xp_radiant'] - data_train['xp_dire']
data_train['dif_gold'] = data_train['gold_radiant'] - data_train['gold_dire']
data_train['dif_items'] = data_train['items_radiant'] - data_train['items_dire']
data_train['dif_lh'] = data_train['lh_radiant'] - data_train['lh_dire']
data_train.fillna(0, inplace=True)

data_train.drop(columns=['xp_radiant','xp_dire','gold_radiant','gold_dire','items_radiant','items_dire','lh_radiant','lh_dire'],inplace=True)

In [157]:
data_test['dif_xp'] = data_test['xp_radiant'] - data_test['xp_dire']
data_test['dif_gold'] = data_test['gold_radiant'] - data_test['gold_dire']
data_test['dif_items'] = data_test['items_radiant'] - data_test['items_dire']
data_test['dif_lh'] = data_test['lh_radiant'] - data_test['lh_dire']
data_test.fillna(0, inplace=True)

data_test.drop(columns=['xp_radiant','xp_dire','gold_radiant','gold_dire','items_radiant','items_dire','lh_radiant','lh_dire'],inplace=True)

In [20]:
# data_train['dif_xp'] = data_train['dif_xp'].apply(lambda x: 1 if x > 0 else 0)
# data_train['dif_gold'] = data_train['dif_gold'].apply(lambda x: 1 if x > 0 else 0)
# data_train['dif_items'] = data_train['dif_items'].apply(lambda x: 1 if x > 0 else 0)
# data_train['dif_lh'] = data_train['dif_lh'].apply(lambda x: 1 if x > 0 else 0)



In [39]:
from sklearn.model_selection import train_test_split

X = data_train.drop(['mid','radiant_won'],axis=1)
y = data_train['radiant_won']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.preprocessing import StandardScaler, MinMaxScaler

# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [45]:

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200, random_state=52, max_depth=6)
clf.fit(X_train, y_train)
clf.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report

y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))

y_pred_proba = clf.predict_proba(X_test)
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, y_pred_proba[:,1])


0.6360360360360361
              precision    recall  f1-score   support

           0       0.63      0.59      0.61      2383
           1       0.64      0.68      0.66      2612

    accuracy                           0.64      4995
   macro avg       0.64      0.63      0.63      4995
weighted avg       0.64      0.64      0.64      4995



0.6734627745406944

In [308]:
X = scaler.transform(data_test.drop('mid',axis=1))

y_pred = clf.predict_proba(X)

In [304]:
submission = pd.DataFrame({'mid':test['mid'], 'radiant_won':y_pred[:,1]})
submission.to_csv('submission.csv', index=False)

In [315]:
data_train.fillna(0, inplace=True)

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print(accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))

y_pred_proba = logreg.predict_proba(X_test)
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, y_pred_proba[:,1])


0.6394394394394395
              precision    recall  f1-score   support

           0       0.62      0.62      0.62      2383
           1       0.65      0.66      0.66      2612

    accuracy                           0.64      4995
   macro avg       0.64      0.64      0.64      4995
weighted avg       0.64      0.64      0.64      4995



0.674842185490769

In [ ]:
# импортируем метод опорных векторов
from sklearn.svm import SVC, LinearSVC, NuSVC, SVR, NuSVR, OneClassSVM, LinearSVR, RBFSVR


In [ ]:
# SVM
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report


# Задание сетки параметров для перебора
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear'],
    'class_weight': ['balanced']
}

# Создание экземпляра модели
svc = SVC()

# Подбор гиперпараметров
grid_search = GridSearchCV(svc, param_grid=param_grid, scoring='f1', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Вывод результатов подбора гиперпараметров
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Получение предсказаний на тестовой выборке и вывод отчета о классификации
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))


In [158]:
merged_data = pd.merge(data_train, heroes, on='mid', how='inner')
player_columns = [f'player_{i}' for i in range(10)]
# Выполним one-hot энкодинг для столбцов игроков
encoded_players = pd.get_dummies(merged_data[player_columns], columns=player_columns, prefix=player_columns)
# Объединим энкодированные данные с остальными столбцами
# Убираем исходные столбцы игроков из merged_data и добавляем вместо них закодированные
final_data = pd.concat([merged_data.drop(columns=player_columns), encoded_players], axis=1)


In [159]:
merged_data_test = pd.merge(data_test, heroes, on='mid', how='inner')
player_columns = [f'player_{i}' for i in range(10)]
# Выполним one-hot энкодинг для столбцов игроков
encoded_players_test = pd.get_dummies(merged_data_test[player_columns], columns=player_columns, prefix=player_columns)
# Объединим энкодированные данные с остальными столбцами
# Убираем исходные столбцы игроков из merged_data и добавляем вместо них закодированные
final_data_test = pd.concat([merged_data_test.drop(columns=player_columns), encoded_players_test], axis=1)

In [160]:
# 1. Найдем общие колонки
common_columns = set(final_data.columns).intersection(set(final_data_test.columns))

# 2. Колонки, которые есть в data, но нет в data_test
only_in_data = set(final_data.columns) - set(final_data_test.columns)

# 3. Колонки, которые есть в data_test, но нет в data
only_in_data_test = set(final_data_test.columns) - set(final_data.columns)

print("Общие колонки:", common_columns)
print("Только в data:", only_in_data)
print("Только в data_test:", only_in_data_test)

Общие колонки: {'player_2_13', 'player_6_44', 'player_7_28', 'player_1_50', 'player_9_85', 'player_7_52', 'player_7_58', 'player_7_96', 'player_6_36', 'player_5_45', 'player_9_86', 'player_6_27', 'player_2_56', 'player_3_47', 'player_4_33', 'player_0_88', 'count_radiant_event_type_0', 'player_3_62', 'player_2_17', 'count_radiant_event_type_4', 'player_5_16', 'count_radiant_event_type_3', 'player_3_37', 'player_3_10', 'player_6_70', 'player_7_86', 'player_8_19', 'player_9_106', 'player_9_4', 'player_9_55', 'player_6_85', 'player_9_103', 'player_4_60', 'count_radiant_event_type_5', 'player_3_24', 'player_4_94', 'player_3_48', 'player_4_109', 'player_7_84', 'player_8_46', 'player_7_97', 'player_5_98', 'player_9_18', 'player_5_12', 'player_1_83', 'player_7_66', 'player_0_18', 'player_5_87', 'player_5_104', 'player_3_25', 'player_4_1', 'player_6_16', 'player_9_26', 'player_9_75', 'player_6_13', 'player_0_42', 'player_3_22', 'player_4_13', 'player_3_78', 'player_0_62', 'player_4_8', 'player_

In [161]:
final_data.drop(columns=['player_6_0','player_5_0', 'player_3_0'], inplace=True)
final_data_test.drop(columns=['player_1_0'], inplace=True)

In [133]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform

# Разделим данные на признаки (X) и целевую переменную (y)
X = final_data.drop(['mid','radiant_won'],axis=1)
y = final_data['radiant_won']

# Стандартизация признаков
scaler = StandardScaler()

# Модель логистической регрессии
logreg = LogisticRegression(max_iter=1000, random_state=42)

# Отбор признаков на основе логистической регрессии
selector = SelectFromModel(estimator=logreg)

# Pipeline для стандартизации, отбора признаков и логистической регрессии
pipeline = Pipeline([
    ('scaler', scaler), 
    ('selector', selector), 
    ('logreg', logreg)
])

# Задаем сетку гиперпараметров для рандомного поиска
param_dist = {
    'selector__max_features': [5, 10, 20, 30, 50],  # Количество признаков для отбора
    'selector__threshold': [0.1, 0.2, 0.3, 0.4, 'mean', 'median'],  # Порог отбора
    'logreg__C': uniform(0.001, 10),               # Инверсия параметра регуляризации
    'logreg__penalty': ['l1', 'l2', 'elasticnet', 'none'],  # Тип регуляризации
    'logreg__solver': ['saga'],                    # Поддержка l1 и elasticnet
    'logreg__l1_ratio': uniform(0, 1)              # Для elasticnet регуляризации
}

# Настройка RandomizedSearchCV
random_search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_dist,
    n_iter=50,             # Количество комбинаций гиперпараметров
    scoring='roc_auc',     # Оптимизация по метрике ROC AUC
    cv=5,                  # Кросс-валидация с 5 фолдами
    random_state=42,       # Для воспроизводимости
    n_jobs=-1              # Параллельное выполнение
)

# Обучение модели
random_search.fit(X, y)

# Получение наилучших параметров
best_params = random_search.best_params_
best_score = random_search.best_score_

print("Лучшие параметры:", best_params)
print("Лучший ROC AUC:", best_score)


C:\Users\1\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
45 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
13 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\1\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\1\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\1\AppData\Roaming\Python\Python312\site-packages\sklearn

Лучшие параметры: {'logreg__C': 3.746401188473625, 'logreg__l1_ratio': 0.9507143064099162, 'logreg__penalty': 'elasticnet', 'logreg__solver': 'saga', 'selector__max_features': 50, 'selector__threshold': 'mean'}
Лучший ROC AUC: 0.7112161644015618


C:\Users\1\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [163]:

y_pred_proba = random_search.predict_proba(final_data_test.drop('mid',axis=1))[:, 1]

In [139]:
final_data_test

,mid,count_radiant_event_type_0,count_radiant_event_type_1,count_radiant_event_type_2,count_radiant_event_type_3,count_radiant_event_type_4,count_radiant_event_type_5,count_radiant_event_type_6,count_dire_event_type_0,count_dire_event_type_1,...,player_9_101,player_9_102,player_9_103,player_9_104,player_9_105,player_9_106,player_9_107,player_9_108,player_9_109,player_9_110
0,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,7,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,10,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28672,49942,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
28673,49943,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
28674,49943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
28675,49944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [138]:
y_pred_proba.shape

(28677,)

In [164]:
submission = pd.DataFrame({'mid':test['mid'], 'radiant_won':y_pred_proba})
submission.to_csv('submission.csv', index=False)